In [ ]:
import numpy as np
from IPython.display import Audio
import scipy.signal as sig
import soundfile as sf
import matplotlib.pyplot as plt
import mat73
import os
from masp import shoebox_room_sim as srs
from os.path import join as pjoin
import copy
import spaudiopy as spa

In [ ]:
# import my modules (helpers.py where I stored all the functions):
import helpers as hlp
import importlib
importlib.reload(hlp);

In [ ]:
def mono2biSH(mono_sig, sh_rir):
    # Apply audio to SH IR
    # sh_rir: (M, maxSH, 2, 1), which comes from (M, maxSH, nRec, nSrc)
    # mono_sig: (1,D)
    left = sig.fftconvolve(np.tile(mono_sig[0], (121,1)).T, sh_rir[:,:,0,0], 'full', 0)   
    right = sig.fftconvolve(np.tile(mono_sig[0], (121,1)).T, sh_rir[:,:,1,0], 'full', 0)
    return np.array([left, right])

def mono2sh(mono_sig, sh_rir):
    # Apply audio to SH IR
    # sh_rir: (M, maxSH, 1, 1), which comes from (M, maxSH, nRec, nSrc)
    # mono_sig: (1,D)
    head = sig.fftconvolve(np.tile(mono_sig[0], (121,1)).T, sh_rir[:,:,0,0], 'full', 0)   
    return head

def biSH2bin(sh_sig, decoder):
    left = sig.fftconvolve(sh_sig[0], decoder[:,:,0], 'full', 0).sum(1)
    right = sig.fftconvolve(sh_sig[1], decoder[:,:,1], 'full', 0).sum(1)
    return np.array([left,right])

def sh2bin(sh_sig, decoder):
    left = sig.fftconvolve(sh_sig, decoder[:,:,0], 'full', 0).sum(1)
    right = sig.fftconvolve(sh_sig, decoder[:,:,1], 'full', 0).sum(1)
    return np.array([left,right])


In [ ]:
speech_path = 'sounds'
maxlim = 2
ambi_order = 10
fs=48000

In [ ]:
# --------------- DESIGN SCENE ----------------

# --- Room: ---
# room dimensions
room = np.array([5., 5., 2.5]) 
rt60 = np.array([.4])
# Compute absorption coefficients for desired rt60 and room dimensions
abs_walls,rt60_true = srs.find_abs_coeffs_from_rt(room, rt60)
# Small correction for sound absorption coefficients:
if sum(rt60_true-rt60>0.05*rt60_true)>0 :
    abs_walls,rt60_true = srs.find_abs_coeffs_from_rt(room, rt60_true + abs(rt60-rt60_true))
# Generally, we simulate up to RT60:
limits = np.minimum(rt60, maxlim)

# --- Receivers: ---
# position of the center of the head:
head_pos= np.array([room[0]/2, room[1]/2, 1.6]) 
# head rotation [az, el]
head_orient = np.array([0, 0])
# position of two receivers on the head (ears):
ears_pos=hlp.head_2_ku_ears(head_pos,head_orient)
# 3 receivers in total: head center, left ear, right ear
mics = np.array([ears_pos[0], ears_pos[1], list(head_pos)]) 

# --- Source: ---
# 1 source position:
src_angle=90
src_pos= hlp.place_on_circle(head_pos,1,src_angle)
src = np.array(src_pos)


# --------------- COMPUTE ECHOGRAMS ----------------
# Echograms define how many reflections, at what time and from which coordinates:
abs_echograms = srs.compute_echograms_sh(room, src, mics, abs_walls, limits, ambi_order)


# --------------- RENDER ECHOGRAMS ----------------
# based on echograms, RIRs in spherical harmonics are generated for each defined receiver
# here there are 3 receivers: head center, left ear, right ear
band_centerfreqs=np.array([1000])
rirs_sh = srs.render_rirs_sh(abs_echograms, band_centerfreqs, fs)

In [ ]:
# check array dimensions:
print(f"{room.shape=} -> Room dimensions in cartesian coordinates. Dimension = (3) [x, y, z].")
print(f"{abs_walls.shape=} -> Wall absorption coefficients per band. Dimension = (nBands, 6).")
print(f"{limits.shape=} -> Maximum echogram computation time per band.  Dimension = (nBands).")
print(f"{src.shape=} -> Source position in cartesian coordinates. Dimension = (nSrc, 3) [[x, y, z]].")
print(f"{mics.shape=} -> Receiver position in cartesian coordinates. Dimension = (nRec, 3) [[x, y, z]].")
print(f"{abs_echograms.shape=} -> Rendered echograms. Dimension = (nSrc, nRec, nBands)")
print(f"{rirs_sh.shape=} -> Rendered RIR in SH. Dimension = (M, maxSH, nRec, nSrc)")

In [ ]:
hlp.plot_scene_raw(room,mics,src,perspective="xy")

In [ ]:
# -------------- SIGNALS IN SPHERICAL HARMONICS -----------------
# Signal in mono is convolved with RIR in SH generated previously 

# load mono file:
src_sig_mono, src_fs = sf.read(pjoin(speech_path, 'pulse_train.wav'))
# Resample mono audio:
src_sig_mono = sig.resample_poly(src_sig_mono, fs, src_fs)
# change shape from (D,) to (1,D):
src_sig_mono=np.array(src_sig_mono,ndmin=2)

# from mono to spherical harmonics (sh for the head center)
src_sig_sh = mono2sh(src_sig_mono, rirs_sh[:,:,2:3,:])
# from mono to binaural spherical harmonics (sh for each ear)
src_sig_sh_bi = mono2biSH(src_sig_mono, rirs_sh[:,:,0:2,:])


# ---> Note! Functions mono2sh and mono2biSH expect a 2-dim array with a signal and a
# 4-dim array with RIRs, therefore I use the python slicing - to pick the RIRs for 
# the third receiver (here corresponding with the center of the head), I use 
# rirs_sh[:,:,2:3,:] - this picks the correct elements without changing the size of
# the array. I also change shape of the signal from (D,) to (1,D). 

In [ ]:
# check array dimensions:
print(f"{src_sig_mono.shape=}")
print(f"{src_sig_sh.shape=}")
print(f"{src_sig_sh_bi.shape=}")

In [ ]:
# --------------- DECODE FROM SH TO BINAURAL ----------------

# load bimagls decoders created in matlab
# bimagls takes RIRs in SH of 2 receivers 
decoder_ku_bimag = mat73.loadmat(pjoin('decoders_ord10', 'Ku100_ALFE_Window_sinEQ_bimag.mat'))['hnm']
decoder_ric_bimag = mat73.loadmat(pjoin('decoders_ord10', 'RIC_Front_Omni_ALFE_Window_SinEQ_bimag.mat'))['hnm']

# create a magls decoder with spaudiopy using a sofa file
# magls takes RIRs in SH of 1 receivers 
hrirs = spa.io.load_sofa_hrirs('sofas/RIC_Front_Omni_48000Hz.sofa')
left, rigth, new_fs = spa.process.resample_hrirs(hrirs.left, hrirs.right, hrirs.fs, fs, 8)
hrirs.fs = new_fs
hrirs.update_hrirs(left, rigth)
decoder_oldb_mag = spa.decoder.magls_bin(hrirs, 10)
decoder_oldb_mag=decoder_oldb_mag.T

print(f"{decoder_ku_bimag.shape=}")
print(f"{decoder_ric_bimag.shape=}")
print(f"{decoder_oldb_mag.shape=}")


In [ ]:
print(f"Source at {src_angle}, BimagLS with RIC data base: ")
Audio(biSH2bin(src_sig_sh_bi, decoder_ric_bimag), rate=fs)

In [ ]:
print(f"Source at {src_angle}, BimagLS with Ku100 data base: ")
Audio(biSH2bin(src_sig_sh_bi, decoder_ku_bimag), rate=fs)

In [ ]:
print(f"Source at {src_angle}, magLS with RIC data base: ")
Audio(sh2bin(src_sig_sh, decoder_oldb_mag), rate=fs)